# Ch `02`: Concept `09`

## Using Queues

If you have a lot of training data, you probably don't want to load it all into memory at once. The QueueRunner in TensorFlow is a tool to efficiently employ a queue data-structure in a multi-threaded way.

In [1]:
import tensorflow as tf
import numpy as np

We will be running multiple threads, so let's figure out the number of CPUs:

In [2]:
import multiprocessing
NUM_THREADS = multiprocessing.cpu_count()

Generate some fake data to work with:

In [3]:
xs = np.random.randn(100, 3)
ys = np.random.randint(0, 2, size=100)

Here's a couple concrete examples of our data:

In [4]:
xs_and_ys = zip(xs, ys)
for _ in range(5):
    x, y = next(xs_and_ys)
    print('Input {}  --->  Output {}'.format(x, y))

Input [ 1.10093799  0.93379728 -0.51003898]  --->  Output 0
Input [ 0.50678392 -0.73707906  0.60728424]  --->  Output 0
Input [ 1.22735133 -0.09350287  0.30050392]  --->  Output 0
Input [-0.94552237  0.14116032  0.12328885]  --->  Output 0
Input [ 0.61669352  0.29604105 -1.03617857]  --->  Output 0


Define a queue:

In [5]:
queue = tf.FIFOQueue(capacity=1000, dtypes=[tf.float32, tf.int32])

Set up the enqueue and dequeue ops:

In [6]:
enqueue_op = queue.enqueue_many([xs, ys])
x_op, y_op = queue.dequeue()

Define a QueueRunner:

In [7]:
qr = tf.train.QueueRunner(queue, [enqueue_op] * 4)

Now that all variables and ops have been defined, let's get started with a session:

In [8]:
sess = tf.InteractiveSession()

Create threads for the QueueRunner:

In [9]:
coord = tf.train.Coordinator()
enqueue_threads = qr.create_threads(sess, coord=coord, start=True)

Test out dequeueing:

In [10]:
for _ in range(100):
    if coord.should_stop():
        break
    x, y = sess.run([x_op, y_op])
    print(x, y)
coord.request_stop()
coord.join(enqueue_threads)

[ 1.100938   0.9337973 -0.510039 ] 0
[ 0.5067839  -0.7370791   0.60728425] 0
[ 1.2273513  -0.09350287  0.3005039 ] 0
[-0.94552237  0.14116032  0.12328885] 0
[ 0.6166935   0.29604104 -1.0361786 ] 0
[-0.46848592  0.39985955  0.8962615 ] 0
[0.9295709 0.4174584 1.1835827] 0
[0.03867971 1.8632767  0.35298023] 1
[ 1.3847383 -1.683892   1.2527925] 0
[-0.46043342 -0.13380565  1.6195946 ] 0
[-1.4583324 -1.1823987  1.3123245] 0
[ 0.58261675  0.6815601  -0.2556204 ] 0
[-0.23592715 -0.11537535 -1.3488603 ] 1
[-1.0837636   0.95104545 -1.2623904 ] 1
[-0.7440529  -0.88322514  0.45656666] 0
[ 0.6256331  -0.03097991  0.2791929 ] 0
[ 1.9985876 -0.4805818 -1.2142864] 0
[ 0.59213597  0.72141516 -0.20481913] 0
[-0.19362614 -0.8165396   1.2770112 ] 0
[ 1.4228933 -2.1072707 -2.2259014] 1
[ 1.2638766  1.3474947 -0.778809 ] 1
[ 0.5630879 -0.8346478 -1.0944185] 0
[-1.2291574  -0.20824137  2.597347  ] 1
[-1.992602   -0.74801546 -0.35211262] 0
[-0.32680854 -3.2792034   0.29830015] 1
[ 0.5317047   0.00475504 -1.55